In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from removing_people import *

# Removing people concept

In [3]:
x = State('foo', int, 0)
z = State('bar', int, lambda n: np.random.randint(1,4,n))

p = DynamicPeople(states=[x,z])
p.initialize(3)


In [4]:
x

     Quantity
UID          
0           0
1           0
2           0

In [5]:
x.uids

array([0, 1, 2])

In [6]:
x.values

array([0, 0, 0])

In [7]:
z

     Quantity
UID          
0           1
1           1
2           1

In [8]:
p.grow(3)

In [9]:
x

     Quantity
UID          
0           0
1           0
2           0
3           0
4           0
5           0

In [10]:
z

     Quantity
UID          
0           1
1           1
2           1
3           3
4           3
5           3

## Some indexing operations

Different kinds of indexing

- Single index
- List/array of indices
- Boolean array
- Slices

In [11]:
x[2] = 10
x[3] = 20
x

     Quantity
UID          
0           0
1           0
2          10
3          20
4           0
5           0

In [12]:
x>0

     Quantity
UID          
0       False
1       False
2        True
3        True
4       False
5       False

In [13]:
x[x>0]

     Quantity
UID          
2          10
3          20

In [14]:
z

     Quantity
UID          
0           1
1           1
2           1
3           3
4           3
5           3

In [15]:
z+= 1

In [16]:
z

     Quantity
UID          
0           2
1           2
2           2
3           4
4           4
5           4

In [17]:
z[[1,3,5]] += 10

In [18]:
z

     Quantity
UID          
0           2
1          12
2           2
3          14
4           4
5          14

In [19]:
np.mean(z)

8.0

## Removing people

In [20]:
x

     Quantity
UID          
0           0
1           0
2          10
3          20
4           0
5           0

In [21]:
p.remove([0,1])

In [22]:
x

     Quantity
UID          
2          10
3          20
4           0
5           0

In [23]:
p.remove(2)

In [24]:
x

     Quantity
UID          
3          20
4           0
5           0

In [25]:
x[3]

20

In [26]:
try:
    x[0:3]
except Exception as e:
    print(e)

Slicing not supported - slice the .values attribute by index instead e.g., x.values[0:5], not x[0:5]


In [27]:
x[:]

     Quantity
UID          
3          20
4           0
5           0

In [28]:
x.values[0:3]

array([20,  0,  0])

## cv.true

This example

```python
filter_inds = people.true('hiv')  # indices fo people with HIV
if len(filter_inds):    
    art_inds = filter_inds[hpu.true(people.art[filter_inds])]  # Indices of people on ART
    not_art_inds = filter_inds[hpu.false(people.art[filter_inds])]
    cd4_remaining_inds = hpu.itrue(((people.t - people.date_hiv[not_art_inds]) * dt) < people.dur_hiv[not_art_inds], not_art_inds)  # Indices of people not on ART who have an active infection
```

illustrates typical usage of the true() function - need to chain indexing i.e., subsequently index `filter_inds` rather than the original `People`.

In [29]:
p.grow(10)

In [30]:
z

     Quantity
UID          
3          14
4           4
5          14
6           1
7           1
8           1
9           1
10          1
11          2
12          1
13          1
14          2
15          3

In [31]:
z>1

     Quantity
UID          
3        True
4        True
5        True
6       False
7       False
8       False
9       False
10      False
11       True
12      False
13      False
14       True
15       True

In [32]:
def true(x):
    return x.uids[np.nonzero(x)].ravel()

In [33]:
gt_1 = true(z>1)
gt_1

array([ 3,  4,  5, 11, 14, 15])

In [34]:
gt_1_lt_3 = true(z[gt_1]<3)
gt_1_lt_3

array([11, 14])

Idea is we would be able to write things like

```python
hiv_uids = ss.true(hiv)
art_uids = ss.true(art[hiv_uids])
not_art_uids = ss.false(art[hiv_uids])
```

## Distributed states

- Define a state anywhere
- Connect it to a people instance at initialization

In [46]:
class Intervention():
    def __init__(self):
        self.vaccinated = State('day_vaccinated',bool)
    def initialize(self, people):
        # nb. in reality this will take in a Sim and use sim.people
        self.vaccinated.initialize(people)
        

In [47]:
my_intervention = Intervention()
my_intervention.vaccinated

<State day_vaccinated (uninitialized)>

In [48]:
len(p)

18

In [49]:
my_intervention.initialize(p)

In [50]:
my_intervention.vaccinated

     Quantity
UID          
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False

In [51]:
my_intervention.vaccinated.values[0:10] = True

In [52]:
my_intervention.vaccinated

     Quantity
UID          
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False

In [53]:
p.grow(5)

In [54]:
my_intervention.vaccinated

     Quantity
UID          
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False

## Stateful modules

If the `State` object is itself stateful and the modules can be stateful too, then we don't need

```
class HIV(module):
    def __init__(self):
        self.states = [
            State('susceptible', bool, True),
            State('infected', bool, False),
            State('ti_infected', float, 0),
            State('ti_dead', float, np.nan),
            ]
```

instead we can actually just have

```
class HIV(module):
    def __init__(self):
        self.sus = State('susceptible', bool, True)
        self.infected = State('infected', bool, False)
        self.ti_infected = State('ti_infected', float, 0)
        self.ti_dead = State('ti_dead', float, np.nan)

```

Then these states can be initialized in-place, and referenced directly. So instead of


```
class HIV(module):
    def set_prognoses(self, sim, uids):
        sim.people[self.name].susceptible[uids] = False
        sim.people[self.name].infected[uids] = True
        sim.people[self.name].ti_infected[uids] = sim.ti
```

we could have

```
class HIV(module):
    def set_prognoses(self, sim, uids):
        self.susceptible[uids] = False
        self.infected[uids] = True
        self.ti_infected[uids] = sim.ti
```

That doesn't preclude ALSO having a reference to these same states in `people.hiv.susceptible`. This would mirror having `sim.pars.hiv` mirror `HIV.pars`

# Class structure

- UID map and UIDs need to be dynamic, but should be accessed by index rather than UID (`DynamicView`)
- States need to be dynamic and accessed by UID (`State(FusedArray)` containing a `DynamicView` where the `FusedArray` values reference the `DynamicView`'s `_view`)
- Indexing states need to return a container accessed by UID but with different UIDs to the people, and does not need to be dynamic (`FusedArray`)
